In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio

In [3]:
# Game variables
board = [[''] * 3 for _ in range(3)]
player = None
ai = None
game_over = False

# Widgets
buttons = [[widgets.Button(description='', layout=widgets.Layout(width='60px', height='60px', font_size='20px')) for _ in range(3)] for _ in range(3)]
output = widgets.Output()
restart_btn = widgets.Button(description='Restart Game', button_style='info')
choice = widgets.ToggleButtons(options=['X', 'O'], description='You play as:')

# Layout
grid = widgets.GridBox(
    children=[btn for row in buttons for btn in row],
    layout=widgets.Layout(
        grid_template_columns="repeat(3, 60px)",
        grid_gap='5px',
        justify_content='center'
    )
)

display(choice, grid, output, restart_btn)
restart_btn.layout.display = 'none'

ToggleButtons(description='You play as:', options=('X', 'O'), value='X')

GridBox(children=(Button(layout=Layout(height='60px', width='60px'), style=ButtonStyle()), Button(layout=Layou…

Output()

Button(button_style='info', description='Restart Game', style=ButtonStyle())

In [5]:
def check_winner(b):
    for i in range(3):
        if b[i][0] == b[i][1] == b[i][2] != '':
            return b[i][0]
        if b[0][i] == b[1][i] == b[2][i] != '':
            return b[0][i]
    if b[0][0] == b[1][1] == b[2][2] != '':
        return b[0][0]
    if b[0][2] == b[1][1] == b[2][0] != '':
        return b[0][2]
    if all(cell != '' for row in b for cell in row):
        return 'Tie'
    return None

def minimax(b, is_ai):
    winner = check_winner(b)
    if winner == ai:
        return 1
    elif winner == player:
        return -1
    elif winner == 'Tie':
        return 0
    
    if is_ai:
        best = -float('inf')
        for i in range(3):
            for j in range(3):
                if b[i][j] == '':
                    b[i][j] = ai
                    score = minimax(b, False)
                    b[i][j] = ''
                    best = max(best, score)
        return best
    else:
        best = float('inf')
        for i in range(3):
            for j in range(3):
                if b[i][j] == '':
                    b[i][j] = player
                    score = minimax(b, True)
                    b[i][j] = ''
                    best = min(best, score)
        return best

def ai_move():
    best_score = -float('inf')
    move = None
    for i in range(3):
        for j in range(3):
            if board[i][j] == '':
                board[i][j] = ai
                score = minimax(board, False)
                board[i][j] = ''
                if score > best_score:
                    best_score = score
                    move = (i, j)
    return move

In [7]:
def update_buttons():
    for i in range(3):
        for j in range(3):
            buttons[i][j].description = board[i][j]
            buttons[i][j].disabled = (board[i][j] != '' or game_over)

def show_result(winner):
    global game_over
    game_over = True
    update_buttons()
    with output:
        clear_output()
        if winner == 'Tie':
            print("It's a tie!")
        else:
            print(f"{winner} wins!")
    restart_btn.layout.display = 'inline-flex'

In [9]:
def on_click(i, j):
    global game_over
    if game_over or board[i][j] != '':
        return

    board[i][j] = player
    update_buttons()

    winner = check_winner(board)
    if winner:
        show_result(winner)
        return

    move = ai_move()
    if move:
        board[move[0]][move[1]] = ai
        update_buttons()

    winner = check_winner(board)
    if winner:
        show_result(winner)

In [11]:
def start_game(change=None):
    global board, player, ai, game_over
    player = choice.value
    ai = 'O' if player == 'X' else 'X'
    board = [[''] * 3 for _ in range(3)]
    game_over = False
    restart_btn.layout.display = 'none'
    with output:
        clear_output()
    update_buttons()
    
    if ai == 'X':
        move = ai_move()
        if move:
            board[move[0]][move[1]] = ai
            update_buttons()

In [13]:
for i in range(3):
    for j in range(3):
        buttons[i][j].on_click(lambda btn, x=i, y=j: on_click(x, y))

restart_btn.on_click(lambda b: start_game())
choice.observe(start_game, names='value')

start_game()